In [30]:
import os
import openai
import subprocess
from dotenv import load_dotenv
import json
import math




def execute_python(file_path):
    # Run the Python file
    result = subprocess.run(['python', file_path], capture_output=True, text=True)

    # Check if the execution was successful
    if result.returncode == 0:
        # Get the output from the executed file
        output = result.stdout.strip()
        #print("Output:", output)
    else:
        # Print the error message if the execution failed
        output = result.stderr.strip()
        #print("Error:", error)
    return output


# Function query GPT-4 with function
def get_response_from_gpt4_funct(sys, usr, assist, tmp=0):
    response = openai.ChatCompletion.create(
    model="gpt-4-0613", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
    #model="gpt-3.5-turbo",
    temperature = tmp,
    messages=[
        {"role": "system", "content": sys},
        {"role": "user", "content": usr},
        {"role": "assistant", "content": assist}
    ],

    functions = [
            {
                "name": "execute_python",
                "description": "Execute the Python code generated by GPT",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "pythonscript": {
                            "type": "string",
                            "description": "The Python script containing the computation generated by GPT. ",
                        }
                    },
                    "required": ["pythonscript"],
                },
            }
        ]    
    
    )
    #res = (response['choices'][0]['message']['content'])
    return response

def get_response_from_gpt4(sys, usr, assist, tmp=0):
    response = openai.ChatCompletion.create(
    model="gpt-4", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
    #model="gpt-3.5-turbo",
    temperature = tmp,
    messages=[
        {"role": "system", "content": sys},
        {"role": "user", "content": usr},
        {"role": "assistant", "content": assist}
    ]    
    
    )
    #res = (response['choices'][0]['message']['content'])
    return response

load_dotenv("C:\Projrcts\Python\MSJupiter\myenv.env")
api_key = os.getenv("API_KEY")
openai.api_key = api_key

# Sample of queries
#qry = "How much is 2 squares."
#qry = "How many characters are in the following text. Text: 123456 8910 12131415"
#qry = "What was the day of the week corresponding to 06/20/1951."
qry = "How much is e power to the sin of 0.15" 
assist = "query: " + qry

# This was used to compare direct results from GPT4 with the ones obtained by the function. 
# In many cases GPT4 did a decent job at computing by itself. 
# sys = "You are a good calculator" 
# usr = "Calculate the following query." 
# response = get_response_from_gpt4(sys, usr, assist) 
# res = response['choices'][0]['message']['content'] 
# print("Initial Response", res) 
 
sys = "You are an outstanding programmer. You will be able to write a Python script to perform a required computation. You will also know that math trigonometric functions expect inputs in radians so you must convert any input to sin, cos, etc using math.radians first. Never input any number to sin cos etc without converting it to radians" 
usr = "Analyze the following query. If you need to perform any computation generate a Python script and use it as a parameter for the function execute_python. The result should be placed in a variable called result." 
 

response = get_response_from_gpt4_funct(sys, usr, assist)
response_message = response["choices"][0]["message"]
if response_message.get("function_call"):
    function_name = response_message["function_call"]["name"]
    print("function_name:", function_name)
    function_args = json.loads(response_message["function_call"]["arguments"])
    
    # Write the file with the generated script
    text = function_args['pythonscript']
    lines = text.split('\n')
    # Write the lines to the file
    with open('data/add_numbers.py', 'w') as file:
        for line in lines:
            file.write(line + '\n')
        file.write('print(result)' + '\n')
    

    #Specify the path to the Python file you want to execute
    file_path = 'data/add_numbers.py'
    output = execute_python(file_path)
    print("Output from the function:", output)

    # create a second query with the result of the first one 
    sys = "You are an excellent calculator but, in this case, if there is any calculation to be made to compute the answer you should not calculate it but get the result from the following text." 
    #output = "Monday" #used to test by generating false answers 
    usr = "Text: Result you need to use = " + output + ". Use this text to provide an answer to the query."     
    
    response = get_response_from_gpt4(sys, usr, assist)
    res = response['choices'][0]['message']['content']
    print("Final Response", res)
else:
    Print("No computation was discovered.")

function_name: execute_python
Output from the function: 1.002621420818084
Final Response The result is 1.002621420818084.
